## NYC Car Crash Data 2016-2021 

<b><i>Dataset exploration, cleaning & feature engineering 

Data source: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95<br>
Notebook by Alan Grunberg, https://github.com/alandavidgrunberg

<b>loading libraries

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

<b>loading original data from CSV into Pandas DataFrame

In [2]:
df_orig = pd.read_csv("data/motor_vehicle_collisions_original.csv")

In [3]:
df = df_orig.copy()

In [4]:
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765.0,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547.0,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903.0,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314.0,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609.0,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199042 entries, 0 to 1199041
Data columns (total 29 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   CRASH DATE                     1199042 non-null  object 
 1   CRASH TIME                     1199038 non-null  object 
 2   BOROUGH                        776108 non-null   object 
 3   ZIP CODE                       775897 non-null   float64
 4   LATITUDE                       1090977 non-null  float64
 5   LONGITUDE                      1090977 non-null  float64
 6   LOCATION                       1090977 non-null  object 
 7   ON STREET NAME                 906899 non-null   object 
 8   CROSS STREET NAME              599549 non-null   object 
 9   OFF STREET NAME                284620 non-null   object 
 10  NUMBER OF PERSONS INJURED      1199020 non-null  float64
 11  NUMBER OF PERSONS KILLED       1199007 non-null  float64
 12  NUMBER OF PEDE

<b>dropping unneeded columns

In [6]:
df = df.drop(columns = ['COLLISION_ID','BOROUGH','ON STREET NAME','CROSS STREET NAME','OFF STREET NAME',\
                        'NUMBER OF PERSONS INJURED','NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF CYCLIST INJURED',\
                        'NUMBER OF MOTORIST INJURED'])

<b> dropping rows with zero fatalities

In [7]:
df = df[df['NUMBER OF PERSONS KILLED'] != 0]

In [8]:
df = df.dropna(subset=['NUMBER OF PERSONS KILLED'])

<b>dropping rows where fatality aggregations don't add up

In [9]:
df['calc_total_killed'] = df['NUMBER OF PEDESTRIANS KILLED'] + df['NUMBER OF CYCLIST KILLED'] + df['NUMBER OF MOTORIST KILLED']
df = df[df['calc_total_killed'] == df['NUMBER OF PERSONS KILLED']]


In [10]:
df = df.drop(columns = ['calc_total_killed'])

<b>dropping rows with missing latitude/longitude values

In [11]:
df = df.dropna(subset=['LATITUDE','LONGITUDE','LOCATION'],how='all')

In [12]:
df = df[df['LATITUDE'] != 0]
df = df[df['LONGITUDE'] != 0]

<b>dropping rows with missing vehicle type/contributing factor information

In [13]:
df = df.dropna(subset=['VEHICLE TYPE CODE 1','VEHICLE TYPE CODE 2','VEHICLE TYPE CODE 3','VEHICLE TYPE CODE 4',\
                       'VEHICLE TYPE CODE 5','CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2',\
                       'CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4',\
                       'CONTRIBUTING FACTOR VEHICLE 5'], how='all')

<b>filling in unspecified vehicle type values

In [14]:
df.is_copy = False
df['VEHICLE TYPE CODE 1'].fillna('Unspecified', inplace=True)


<b>converting crash date and time info to Pandas datetime objects

In [15]:
df['CRASH TIME'] = df['CRASH TIME'].str.zfill(5)

In [16]:
df['CRASH DATE'] = df['CRASH DATE'].str.replace('/','')
df['CRASH TIME'] = df['CRASH TIME'].str.replace(':','')

In [17]:
df['date time'] = df['CRASH DATE'] + df['CRASH TIME']

In [18]:
df['CRASH DATETIME'] = pd.to_datetime(df['date time'], format='%m%d%Y%H%M')

In [19]:
df = df.drop(columns=['date time','CRASH DATE','CRASH TIME'])

<b>sorting rows by datetime

In [20]:
df = df.sort_values(by=['CRASH DATETIME'])

<b> slicing off partial 2015 and 2022 rows to leave full years from 2016-2021

In [21]:
df = df[1:]


In [22]:
df = df[:-127]


In [23]:
df = df.reset_index(drop=True)

<b>getting zip code for each row by reverse geocoding coordinates

In [24]:
geolocator = Nominatim(user_agent="nyc crash data")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1, max_retries=3)

In [25]:
zipcode_list = []
for lat, lon in zip(df['LATITUDE'],df['LONGITUDE']):
    try:
        zipcode_list.append(reverse(f"{lat},{lon}").raw['address']['postcode'])
    except AttributeError:
        zipcode_list.append(None)

In [26]:
zipcode_series = pd.Series(zipcode_list)

In [27]:
df['ZIP'] = zipcode_series


In [28]:
df = df.drop(columns=['ZIP CODE'])


In [29]:
df.head()

,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CRASH DATETIME,ZIP
0,40.883861,-73.855885,"(40.8838608, -73.8558854)",1.0,0.0,0.0,1.0,Traffic Control Disregarded,Unspecified,Unspecified,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,2016-01-04 05:20:00,10466
1,40.739903,-73.972845,"(40.7399029, -73.9728449)",1.0,1.0,0.0,0.0,Passenger Distraction,NaN,NaN,NaN,NaN,PASSENGER VEHICLE,NaN,NaN,NaN,NaN,2016-01-08 00:35:00,10010
2,40.839225,-73.915089,"(40.8392249, -73.9150891)",1.0,1.0,0.0,0.0,Driver Inattention/Distraction,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,PASSENGER VEHICLE,VAN,SPORT UTILITY / STATION WAGON,NaN,NaN,2016-01-08 23:14:00,10456
3,40.718303,-73.987396,"(40.7183032, -73.9873956)",1.0,1.0,0.0,0.0,Unspecified,NaN,NaN,NaN,NaN,LARGE COM VEH(6 OR MORE TIRES),NaN,NaN,NaN,NaN,2016-01-12 13:29:00,10002
4,40.794052,-73.970367,"(40.7940523, -73.9703673)",1.0,1.0,0.0,0.0,Unspecified,NaN,NaN,NaN,NaN,MOTORCYCLE,NaN,NaN,NaN,NaN,2016-01-14 17:46:00,10025


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1274 entries, 0 to 1273
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   LATITUDE                       1274 non-null   float64       
 1   LONGITUDE                      1274 non-null   float64       
 2   LOCATION                       1274 non-null   object        
 3   NUMBER OF PERSONS KILLED       1274 non-null   float64       
 4   NUMBER OF PEDESTRIANS KILLED   1274 non-null   float64       
 5   NUMBER OF CYCLIST KILLED       1274 non-null   float64       
 6   NUMBER OF MOTORIST KILLED      1274 non-null   float64       
 7   CONTRIBUTING FACTOR VEHICLE 1  1270 non-null   object        
 8   CONTRIBUTING FACTOR VEHICLE 2  557 non-null    object        
 9   CONTRIBUTING FACTOR VEHICLE 3  177 non-null    object        
 10  CONTRIBUTING FACTOR VEHICLE 4  84 non-null     object        
 11  CONTRIBUTING FACT

In [32]:
df.to_csv("data/motor_vehicle_collisions_prepared.csv")